<a href="https://colab.research.google.com/github/alexisjkim/conformal_prediction_limitations/blob/main/Conformal_Prediction_Limitations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [238]:
# imports

import numpy as np
import matplotlib.pyplot as plt
import time
import math


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, ConcatDataset, random_split
import torchvision


In [239]:
# loading data and splitting into train, calibration, test sets

batch_size = 128

# this dataset has the training data from MNIST; will be split into training and calibration sets
mnist_train_set = torchvision.datasets.MNIST(root='./datasets/',
                                           train=True,
                                           transform=torchvision.transforms.ToTensor(),
                                           download=True)



# this dataset has the test data from MNIST
mnist_test_dataset = torchvision.datasets.MNIST(root='./datasets',
                                          train=False,
                                          transform=torchvision.transforms.ToTensor())


train_percentage = 0.8 # this percentage of the training data set stays in the train set; the rest becomes part of the calibration set

train_size = int(train_percentage *len(mnist_train_set))
calibration_size = len(mnist_train_set) - train_size

mnist_train_set, mnist_cal_set = random_split(mnist_train_set, [train_size, calibration_size])

# Data loader
mnist_train_loader = torch.utils.data.DataLoader(dataset=mnist_train_set,
                                           batch_size=batch_size,
                                           shuffle=True, drop_last=True)

mnist_cal_loader = torch.utils.data.DataLoader(dataset=mnist_cal_set,
                                           batch_size=batch_size,
                                           shuffle=True, drop_last=True)

# We use drop_last=True to avoid the case where the data / batch_size != int

mnist_test_loader = torch.utils.data.DataLoader(dataset=mnist_test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

print("SIZES OF DATASETS: ")
print("training set: ", len(mnist_train_loader.dataset))
print("calibration set: ", len(mnist_cal_loader.dataset))
print("testing set: ", len(mnist_test_loader.dataset))

SIZES OF DATASETS: 
training set:  48000
calibration set:  12000
testing set:  10000


In [240]:
# class for our neural network

class TwoLayerNetPiped(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        Parameters:
            D_in - dimensions of inputs
            H - number of hidden units per layer
            D_out - dimensions of outputs
        """
        # initialzing the parent object (important!)
        super(TwoLayerNetPiped, self).__init__()
        # Create a pipeline - a sequence of layers
        self.pipe = torch.nn.Sequential(
            torch.nn.Linear(D_in, H),
            torch.nn.ReLU(),
            torch.nn.Linear(H, D_out))

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        Parameters:
            x - tensor of inputs (shape: [BATCH_SIZE, D_in])
        """
        return self.pipe(x)

In [241]:
# Setting up the model

# hyper-parameters:
num_epochs = 1
learning_rate = 0.001

# Device configuration, as before
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# create model, send it to device
model = TwoLayerNetPiped(D_in=28*28, H=256, D_out=10).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [242]:
# Train the model

def train_model(loader):

    model.train()  # training mode
    total_step = len(loader)
    start_time = time.time()
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loader):
            # each i is a batch of 128 samples
            images = images.to(device).view(batch_size, -1)  # represent images as column vectors
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize - ALWAYS IN THIS ORDER!
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i + 1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Time: {:.4f} secs'
                    .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(), time.time() - start_time))

In [243]:
train_model(mnist_train_loader)

Epoch [1/1], Step [100/375], Loss: 0.3462, Time: 0.4072 secs
Epoch [1/1], Step [200/375], Loss: 0.2558, Time: 0.7771 secs
Epoch [1/1], Step [300/375], Loss: 0.1883, Time: 1.1527 secs


In [244]:
# Test the model

def test_model(model, test_loader):

  model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance), or use:
  with torch.no_grad(): # "don't keep track of the gradients" ,can also use .detach()
      correct = 0
      total = 0
      for images, labels in test_loader:
          images = images.to(device).view(images.size(0), -1) #image.size(0) returns batch size
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


test_model(model, mnist_test_loader)

Test Accuracy of the model on the 10000 test images: 93.9 %


In [245]:
def get_probabiliites_and_predictions(probs_list, preds_list, loader):
    probs_list = []
    preds_list = []
    with torch.no_grad():
        for images, _ in loader:
            images = images.to(device).view(images.size(0), -1)
            outputs = model(images)
            probabilities = torch.nn.functional.softmax(outputs, dim = 1) #1 corresponds to columns
            #the prediction outputs the index of what it thinks the class is
            #the ommited term is the value
            _, predictions = torch.max(outputs.data, 1) #get the index of the highest output
            # Append to lists
            preds_list.extend(predictions.cpu().numpy())
            probs_list.extend(probabilities.cpu().numpy())

    return probs_list, preds_list

Conformal prediction starts here

In [246]:
#Calibration
def get_cal_probs(loader):
  cal_probs = []
  cal_preds = []
  cal_probs, cal_preds = get_probabiliites_and_predictions(cal_probs, cal_preds, loader)
  print("cal_probs:", cal_probs[:5])
  print("cal_preds:", cal_preds[:5])

  cal_scores = []
  for prob, true_label in zip(cal_probs, cal_preds): #prob is the probability and true_label is index of pred
    true_class_prob = prob[true_label] #the corresponding lists with their prob function getting the most predicted class
    cal_scores.append(1 - true_class_prob) #s_i score

  cal_scores = np.array(cal_scores)
  sorted_scores = np.sort(cal_scores) #probabilities

  return sorted_scores


In [247]:


def get_quantile(scores, alpha, loader):
  n = 0
  for _, labels in loader:
    n += labels.size(0)
    
  q_level = math.ceil((1 - alpha) * (n + 1)) / n

  print(f"Adjusted quantile level: {q_level}")
  return np.percentile(scores, (1 - alpha) * 100)




In [248]:
alpha = 0.05
sorted_scores = get_cal_probs(mnist_cal_loader)
threshold = get_quantile(sorted_scores, alpha, mnist_cal_loader) #for the calibration set
print(threshold)

cal_probs: [array([9.3085878e-04, 3.7398211e-05, 1.8736504e-03, 1.0926040e-04,
       1.5031201e-04, 2.6068799e-03, 9.9416536e-01, 1.2302830e-05,
       4.2266569e-05, 7.1714807e-05], dtype=float32), array([1.8455107e-02, 7.8753212e-05, 4.5731825e-01, 2.7203837e-02,
       2.4777562e-03, 9.0805935e-03, 1.1539598e-04, 2.7062741e-01,
       1.5590148e-02, 1.9905274e-01], dtype=float32), array([2.8487886e-05, 2.5531068e-05, 2.3621672e-03, 1.7810137e-03,
       7.7062019e-04, 4.1201366e-03, 4.3087686e-05, 9.7918848e-07,
       9.9034470e-01, 5.2335719e-04], dtype=float32), array([4.92926119e-07, 9.46744372e-09, 2.45024476e-05, 8.16601023e-05,
       1.00103435e-07, 9.91209507e-01, 2.35806730e-09, 1.44923656e-10,
       8.68356228e-03, 2.46513935e-07], dtype=float32), array([6.8132962e-05, 3.2042010e-08, 9.3226066e-05, 3.3552597e-06,
       9.9868101e-01, 8.4813546e-05, 1.9283133e-04, 6.5558146e-05,
       1.6285777e-04, 6.4813002e-04], dtype=float32)]
cal_preds: [6, 2, 8, 5, 4]
Adjusted qu

In [249]:
import pandas as pd
#we know the conformal prediciton model takes in probabilities until it reaches the
#the threshold q hat
#the threshold is q hat (that quantile value)

def conformal_prediction(probabilities, threshold):
    predictions = []
    for prob in probabilities:
        sorted_indices = np.argsort(prob)[::-1]
        total = 0.0
        prediction = []
        for i in sorted_indices:
            total += prob[i]
            prediction.append(i)
            if total > 1 - threshold: #we do 1 - threshold because we want to observe the right side as we are adding in ascending order
                break
        predictions.append(prediction)
    return predictions





In [250]:
#evaluation
def evaluate_and_print(observed_labels, conformal_prediction, predicted, start_row, end_row):
  formatted_output = pd.DataFrame({
    'observed labels': observed_labels,
    'confromal prediction set': conformal_prediction,
    'prediction': predicted
  })


  #evaluation metric
  hits = 0
  total = len(observed_labels)
  for i in range(len(observed_labels)):
    conf_pred_row = conformal_prediction[i]
    observed = observed_labels[i]

    if observed in conf_pred_row:
      hits += 1

  print("the prediction was in the set ", hits/total *100, " percent of the time")

  print(formatted_output[start_row:end_row])


In [251]:
#getting observed label functions
def get_observed_labels(loader):
    observed_labels = []
    with torch.no_grad():
        for images, labels in loader:
            observed_labels.extend(labels.cpu().numpy())
    return observed_labels

In [252]:
#evaluating our testing data set

test_observed_labels = get_observed_labels(mnist_test_loader)

test_probs = []
test_preds = []
test_probs, test_preds = get_probabiliites_and_predictions(test_probs, test_preds, mnist_test_loader)


conformal_predictions = conformal_prediction(test_probs, threshold)
print("The conformal prediction accuracy based on training data and testing data from the same distribution is:")
evaluate_and_print(test_observed_labels, conformal_predictions, test_preds, 300, 450)



The conformal prediction accuracy based on training data and testing data from the same distribution is:
the prediction was in the set  95.53  percent of the time
     observed labels confromal prediction set  prediction
300                4                   [6, 1]           6
301                7                      [7]           7
302                1                      [1]           1
303                2                      [2]           2
304                4                      [4]           4
..               ...                      ...         ...
445                6                      [0]           0
446                6                      [6]           6
447                4                      [4]           4
448                9                   [8, 9]           8
449                3                      [3]           3

[150 rows x 3 columns]


Repeating the experiment with test data that doesn't match the calibration set.

In [253]:
# Train the model -- additional training on blurred images without calibration
from torch.utils.data import DataLoader, TensorDataset
import cv2

Functions for Blurring

In [254]:
def blur_images(image):
  return torch.tensor(cv2.blur(image.numpy(), (30, 30)))

In [255]:
#get blurred data
#blur the images
def get_blurred_data(loader):
    blurred_images = []
    blurred_labels = []

    # adding blur to every image in the training set
    with torch.no_grad():
        for images, labels in loader:
            for i in range(len(images)):
                blurred_image = blur_images(images[i])
                blurred_images.append(blurred_image)
                blurred_labels.append(labels[i])

    blurred_images = torch.stack(blurred_images)
    blurred_labels = torch.tensor(blurred_labels)

    return blurred_images, blurred_labels

In [256]:
def create_blur_loader(blurred_images, blurred_labels):
    # Create a TensorDataset and DataLoader for the blurred images
    blurred_dataset = TensorDataset(blurred_images, blurred_labels)
    blurred_loader = DataLoader(blurred_dataset, batch_size=128, shuffle=False)
    return blurred_loader

In [257]:
#testing our model on blurred images

blur_loader = torch.utils.data.DataLoader(dataset=mnist_test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


observed_labels = get_observed_labels(blur_loader)

blurred_images, blurred_labels = get_blurred_data(blur_loader)
blurred_loader = create_blur_loader(blurred_images, blurred_labels)
blur_probs = []
blur_preds = []
blur_probs, blur_preds = get_probabiliites_and_predictions(blur_probs, blur_preds, blurred_loader)
'''''
blur_probs = []
blur_pred = []

# Flatten images for model input
blurred_images = blurred_images.view(len(blurred_images), -1).to(device)

#find probabilities and predictions
with torch.no_grad():
  outputs = model(blurred_images)
  probabilities = torch.nn.functional.softmax(outputs, dim = 1)
  _, blur_pred = torch.max(outputs.data, 1)


blur_preds = blur_pred.cpu().numpy()
blur_probs = probabilities.cpu().numpy()'''


conformal_predictions = conformal_prediction(blur_probs, threshold)
print("This is the conformal prediction accuracy based on training data and testing data from different distributions. \nThe training data contains normal images while the testing data were all blurred:")
evaluate_and_print(observed_labels, conformal_predictions, blur_preds, 10, 200)



This is the conformal prediction accuracy based on training data and testing data from different distributions. 
The training data contains normal images while the testing data were all blurred:
the prediction was in the set  23.09  percent of the time
     observed labels confromal prediction set  prediction
10                 0                [2, 5, 3]           2
11                 6                [5, 3, 2]           5
12                 9                      [3]           3
13                 0                [2, 3, 5]           2
14                 1                      [1]           1
..               ...                      ...         ...
195                3                [2, 3, 5]           2
196                1                      [1]           1
197                6                      [3]           3
198                4                   [3, 7]           3
199                2                      [3]           3

[190 rows x 3 columns]


Training on blurred Images

In [258]:
# Train the model -- additional training on blurred images without calibration


blurred_train_loader = torch.utils.data.DataLoader(dataset=mnist_train_set,
                                          batch_size=batch_size,
                                          shuffle=False)


observed_labels = get_observed_labels(blurred_train_loader)

blurred_images, blurred_labels = get_blurred_data(blurred_train_loader)
blurred_train_loader = create_blur_loader(blurred_images, blurred_labels)
blur_train_probs = []
blur_train_preds = []
blur_probs, blur_preds = get_probabiliites_and_predictions(blur_train_probs, blur_train_preds, blurred_train_loader)


train_model(blurred_train_loader)


Epoch [1/1], Step [100/375], Loss: 1.0233, Time: 0.1720 secs
Epoch [1/1], Step [200/375], Loss: 0.8217, Time: 0.2688 secs
Epoch [1/1], Step [300/375], Loss: 0.5997, Time: 0.3676 secs


In [260]:
#now use original calibration (the conformal predicition with the blurry images)

#creating blurry images again

blur_test_loader = torch.utils.data.DataLoader(dataset=mnist_test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

observed_labels = get_observed_labels(blur_test_loader)

blurred_images, blurred_labels = get_blurred_data(blur_test_loader)
blurred_test_loader = create_blur_loader(blurred_images, blurred_labels)


blur_test_preds = []
blur_test_probs = []

blur_test_probs, blur_test_preds = get_probabiliites_and_predictions(blur_test_probs, blur_test_preds, blurred_test_loader)


conformal_predictions = conformal_prediction(blur_test_probs, threshold)
print("After putting additional training with blurred images with no recalibration:")
evaluate_and_print(observed_labels, conformal_predictions, blur_test_preds, 100, 150)


After putting additional training with blurred images with no recalibration:
the prediction was in the set  89.92  percent of the time
     observed labels confromal prediction set  prediction
100                6                      [6]           6
101                0                      [0]           0
102                5                      [5]           5
103                4                      [4]           4
104                9                      [9]           9
105                9                      [9]           9
106                2                      [0]           0
107                1                      [1]           1
108                9                      [9]           9
109                4                   [9, 4]           9
110                8                   [8, 3]           8
111                7                   [1, 7]           1
112                3                [3, 5, 6]           3
113                9                      [9]        

In [261]:
#Now lets compare. lets calibrate the new blurred images
#Calibration of the blurred images

#blur everyother image in the calibration data


mnist_cal_loader = torch.utils.data.DataLoader(dataset=mnist_cal_set,
                                           batch_size=batch_size,
                                           shuffle=True, drop_last=True)


#blurs every other images
new_cal_images = []
new_cal_labels = []
with torch.no_grad():
    for images, labels in mnist_cal_loader:
        for i in range(len(images)):
          if i % 2 == 0:
            blurred_image = blur_images(images[i])
            new_cal_images.append(blurred_image)
            new_cal_labels.append(labels[i])
          else:
            new_cal_images.append(images[i])
            new_cal_labels.append(labels[i])


new_cal_images = torch.stack(new_cal_images)
new_cal_labels = torch.tensor(new_cal_labels)

new_cal_loader = create_blur_loader(new_cal_images, new_cal_labels)

new_cal_observed_labels = get_observed_labels(new_cal_loader)

alpha = 0.05
sorted_scores = get_cal_probs(new_cal_loader)
new_cal_threshold = get_quantile(sorted_scores, alpha, new_cal_loader) #for the calibration set
print(new_cal_threshold)

#after calibrating get the accuracy
# we calibratted on 50:50 now we want to test on all blurred images

conformal_predictions = conformal_prediction(blur_test_probs, new_cal_threshold)
print("After putting additional training with blurred images with no recalibration:")
evaluate_and_print(observed_labels, conformal_predictions, blur_test_preds, 100, 150)


cal_probs: [array([2.1286048e-03, 1.8863025e-04, 1.1733644e-02, 7.7244453e-03,
       2.2128671e-01, 6.8366039e-03, 2.5394693e-02, 2.1462882e-02,
       1.0846730e-02, 6.9239700e-01], dtype=float32), array([1.59335032e-03, 2.57829106e-05, 8.73623312e-01, 1.22923955e-01,
       3.75307234e-13, 3.72674986e-04, 4.96868857e-08, 9.28163900e-07,
       1.45993195e-03, 7.50253071e-12], dtype=float32), array([3.8844727e-02, 6.6245675e-01, 1.5717421e-01, 1.7904196e-02,
       4.5091906e-03, 5.2619702e-04, 1.5082151e-02, 3.7710045e-02,
       4.9831405e-02, 1.5961099e-02], dtype=float32), array([9.9998331e-01, 5.6166445e-09, 9.4805796e-07, 9.4981125e-08,
       2.6232028e-15, 1.5659100e-05, 2.3029896e-11, 3.8846411e-09,
       1.1626632e-08, 2.0920380e-15], dtype=float32), array([0.00490593, 0.00654617, 0.01403603, 0.03789134, 0.00795527,
       0.03962396, 0.12203734, 0.00978187, 0.6152176 , 0.14200442],
      dtype=float32)]
cal_preds: [9, 2, 1, 0, 8]
Adjusted quantile level: 0.950100806451612

In [264]:
#sanity check -- what happens if you test with all non-bluured images with 50:50 calibration
test_observed_labels = get_observed_labels(mnist_test_loader)
new_cal_normal_conformal_predictions = conformal_prediction(test_probs, new_cal_threshold)
print("The conformal prediction accuracy based on training data and testing data from the same distribution is:")
evaluate_and_print(test_observed_labels, new_cal_normal_conformal_predictions, test_preds, 300, 450)

The conformal prediction accuracy based on training data and testing data from the same distribution is:
the prediction was in the set  94.12  percent of the time
     observed labels confromal prediction set  prediction
300                4                      [6]           6
301                7                      [7]           7
302                1                      [1]           1
303                2                      [2]           2
304                4                      [4]           4
..               ...                      ...         ...
445                6                      [0]           0
446                6                      [6]           6
447                4                      [4]           4
448                9                      [8]           8
449                3                      [3]           3

[150 rows x 3 columns]


In [ ]:
'''''import cv2



rotate_loader = torch.utils.data.DataLoader(dataset=mnist_test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

observed_labels = []
with torch.no_grad():
    for images, labels in mnist_test_loader:
        observed_labels.extend(labels.cpu().numpy())


def rotate_image(image):
    image = image.numpy().squeeze()  # Convert to numpy array and remove batch dimension
    rows, cols = image.shape
    rotated_image = cv2.warpAffine(image, dsize=None, M=cv2.getRotationMatrix2D((cols / 2, rows / 2), -120, 1))
    return torch.tensor(rotated_image, dtype=torch.float32).unsqueeze(0)  # Add batch dimension back

rotated_images = []
rotated_labels = []

with torch.no_grad():
    for images, labels in mnist_test_loader:
        for i in range(len(images)):
            rotated_image = rotate_image(images[i])
            rotated_images.append(rotated_image)
            rotated_labels.append(labels[i])


rotated_images = torch.stack(rotated_images)
rotated_labels = torch.tensor(rotated_labels)

# Evaluate the model on the rotated test set
rotate_probs = []
rotate_preds = []

# Flatten images for model input
rotated_images = rotated_images.view(len(rotated_images), -1).to(device)


# Evaluate the model on the entire rotated test set at once
with torch.no_grad():
    outputs = model(rotated_images) #
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    _, rotate_preds = torch.max(outputs.data, 1)

rotate_preds = rotate_preds.cpu().numpy()
rotate_probs = probabilities.cpu().numpy()

conformal_predictions = conformal_prediction(rotate_probs, threshold)
print("flip evaluation:")
evaluate_and_print(observed_labels, conformal_predictions, rotate_preds, 10, 200)

'''''


flip evaluation:
the prediction was in the set  23.65  percent of the time
     observed labels confromal prediction set  prediction
10                 0                      [0]           0
11                 6                   [0, 8]           0
12                 9                   [4, 3]           4
13                 0                      [0]           0
14                 1                      [4]           4
..               ...                      ...         ...
195                3                   [4, 6]           4
196                1                      [4]           4
197                6                      [9]           9
198                4                      [4]           4
199                2                      [6]           6

[190 rows x 3 columns]
